<a href="https://colab.research.google.com/github/rmellajr/Projeto_Aula4_ImersaoIA/blob/main/Projeto_Aula05_Imersao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U google-generativeai

In [6]:
import google.generativeai as genai
import pandas as pd
import numpy as np


In [7]:
from google.colab import userdata
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

In [8]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [9]:
#Exemplo de Embedding
title = "A próxima geração de IA para desenvolvedores e Google Workspace"
sample_text = ("Titulo: A próxima geração de IA para desenvolvedores e Google Workspace"
"\n"
"Artigo completo:\n"
"\n"
"Gemini API & Google AI Studio: Uma maneira acessível de explorar e criar protótipos com aplicações de IA generativa")

embeddings = genai.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")
print(embeddings)

{'embedding': [0.050890755, -0.037246283, -0.018431226, 0.018790184, 0.06608615, -0.004846209, -0.027609862, -0.018105436, 0.06159631, 0.06271742, 0.011369126, 0.013108909, -0.046663653, -0.022218306, 0.009782968, -0.023993805, 0.021617359, -0.011871687, -0.035149682, -0.0011378455, 0.008662901, 0.0067902775, -0.029798262, -0.06736514, -0.018991727, 0.021183532, 0.01535126, -0.037774406, -0.03318969, 0.027746424, -0.048998035, 0.0507089, -0.034133896, 0.009643745, -0.036094204, -0.042003963, -0.026732994, -0.044611316, -0.0008435263, -0.0043267272, 0.0053845607, -0.08440501, -0.009712467, 0.025687205, -0.0043043518, -0.021698005, 0.045403242, 0.04396371, 0.014208992, -0.059025764, 0.029370515, 0.022892706, 0.06585143, -0.04319518, -0.0033994205, -0.008611404, 0.023195915, -0.03330959, 0.023971215, 0.00081927417, -0.0012759138, 0.022347605, -0.017549256, 0.05341226, 0.019217424, -0.05777917, -0.043209262, -0.00027597882, 0.010957023, 0.048468057, 0.009249446, 0.011975633, 0.060952824, -

In [10]:
DOCUMENT1 = {
    "title": "Operating the Climate Control System",
    "content": "Your Googlecar has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console.  Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car. Heat: The car will blow warm air into the car. Defrost: The car will blow warm air onto the windshield to defrost it."}
DOCUMENT2 = {
    "title": "Touchscreen",
    "content": "Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the \"Navigation\" icon to get directions to your destination or touch the \"Music\" icon to play your favorite songs."}
DOCUMENT3 = {
    "title": "Shifting Gears",
    "content": "Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions."}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [11]:
df = pd.DataFrame(documents)


In [12]:
model = "models/embedding-001"

In [13]:
def embed_fn(title,text):
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [14]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["title"], row["content"]), axis=1)
df

,title,content,Embeddings
0,Operating the Climate Control System,Your Googlecar has a climate control system th...,"[-0.033361107, -0.021217084, -0.049581926, -0...."
1,Touchscreen,Your Googlecar has a large touchscreen display...,"[0.009660736, -0.030662702, -0.017281422, -0.0..."
2,Shifting Gears,Your Googlecar has an automatic transmission. ...,"[-0.04270796, -0.007160868, -0.03242516, -0.02..."


In [22]:
def gerar_e_buscar_consulta(consulta, base, model):
  embeding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]
  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embeding_da_consulta)
  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["content"]

In [33]:
consulta = "Is there any display in google car?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.


In [30]:
generation_config = {
    "temperature": 0,
    "candidate_count": 1,
    }

In [32]:
prompt = f"Regenerate the text in a funny way, without adding information that does not be part of the text : {trecho}"
model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

Your Googlecar's got a touchscreen so big, it'll make your hamster jealous! It's like a giant smartphone on wheels.

To use it, just poke the screen like a pro. Want to get lost? Tap the "Navigation" icon. Feeling musical? Bop the "Music" icon and jam out to your favorite tunes.

Just remember, don't get too distracted by the touchscreen. You still need to keep your eyes on the road, unless you want to end up in a ditch with a chorus of "Oops, I Did It Again" playing in the background.
